# Ejemplos de algoritmos genéticos
---

Tutorial completo: http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

Referencia DEAP: http://deap.readthedocs.io/en/master/api/tools.html


## Problema OneMax

En este problema de optimización nos encontramos con un vector de 100 valores binarios, por lo que el número de posibles casos es de $2^{100}$. La tarea consiste en encontrar el vector con mayor número de $1$'s.

Las principales clases de DEAP y que vamos a utilizar son:

- <code>base</code> acceso al *toolbox* y a las funciones de *fitness*.
- <code>creator</code> permite crear los tipos (*types*).
- <code>tools</code> acceso a los operadores.
- <code>algorithms</code> prepara las iteraciones de los algoritmos genéticos.

In [ ]:
import random
import numpy
from deap import base, creator, tools, algorithms

### Fitness

La clase "Fitness" proporcionada es una **clase abstracta** que necesita un atributo de pesos para ser funcional. Un "fitness" a minimizar se construye utilizando pesos negativos, mientras que para maximizar debemos coloar pesos positivos. Es posible que la función de fitness incluya varias funciones internas donde unas deban maximizarse y otras minimizarse. Por esta razón el parámetro "weights" es una tupla.

La función *create()* tiene al menos dos argumentos, un nombre para la clase recién creada y una clase base. Cualquier argumento subsiguiente se convierte en un **atributo de la clase**.

### Individuos

El primer individuo que crearemos será una simple lista que contiene flotantes. Para producir este tipo de individuo, necesitamos crear una clase *Individual*, usando el creador, que heredará del tipo de lista estándar y tendrá un atributo fitness.

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

Vemos que <code>Individual</code> es una clase que hereda de <code>list</code> y tiene un método llamado <code>fitness</code>. Podemos crear un individuo <code>ind</code> a partir de <code>creator.Individual</code>, como se muestra a continuación:

In [ ]:
ind = creator.Individual([1, 0, 1, 1, 0])

print(ind)
print(type(ind))
print(type(ind.fitness))

El valor del <code>fitness</code> de un individuo se calculará simplemente sumando todos sus elementos.

In [ ]:
def evalOneMax(individual):
    return sum(individual),

Ahora registraremos varias funciones para crear los atributos de los individuos, los propios individuos y la población. También registraremos las funciones para evaluar los individuos, cruzarlos, mutarlos y seleccionarlos.

In [ ]:
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)

bit = toolbox.attr_bool()
pop = toolbox.population(n=50)

# print("bit is of type %s and has value\n%s" % (type(bit), bit))
# print("ind is of type %s and contains %d bits\n%s" % (type(ind), len(ind), ind))
# print("pop is of type %s and contains %d individuals\n%s" % (type(pop), len(pop), pop))

Por ejemplo, veamos cómo crearíamos un individuo y cómo lo mutaríamos. Observa la línea <code>temp = ind[:]</code>, con este "truco" logramos crear una nueva copia de la lista. Si hubiéramos hecho <code>temp = ind</code>, entonces <code>temp</code> e <code>ind</code> serían el mismo objeto, cosa que no queremos.

In [ ]:
import numpy as np

ind = toolbox.individual()
temp = ind[:]
print(ind)
toolbox.mutate(temp)
print(np.array(ind) - np.array(temp))

print(id(ind))
print(id(temp))

Otra forma de hacer lo mismo pero con el método <code>clone</code> de <code>tolbox</code>.

In [ ]:
mutant = toolbox.clone(ind)
print(mutant is ind)
print(mutant == ind)

### Hall of fame

Si queremos mantener durante toda la evolución del algoritmo los mejores individuos obtenidos hasta el momento, debemos crear un objeto "hall of fame". En este caso, vamos a mantener cuatro. También podemos ir mostrando estadístias a medida que el algoritmo avanza.

In [ ]:
hof = tools.HallOfFame(4)

stats = tools.Statistics(lambda indiv: indiv.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

In [ ]:
print("Best individual is: %s\n with fitness: %s" % (hof[0], hof[0].fitness))

import matplotlib.pyplot as plt

gen, avg, min_, max_ = logbook.select("gen", "avg", "min", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, min_, label="minimum")
plt.plot(gen, max_, label="maximum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

In [ ]:
hof.keys

---

## Problema de la mochila

El **problema de la mochila** es un problema clásico dentro de la IA. Consiste en lo siguiente: existe un número determinado de objetos que tienen un valor y un peso propios. En nuestra mochila solo podemos llevar hasta un peso máximo, por lo tanto, el problema consiste en escoger los objetos que minimicen el peso y maximicen el valor. Es un problema NP-completo.

In [ ]:
import random
import numpy
from deap import base, creator, tools, algorithms

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))  # minimizamos el peso y maximizamos el valor
creator.create("Individual", set, fitness=creator.Fitness)

In [ ]:
IND_INIT_SIZE = 5
MAX_ITEM = 50  # Número máximo de objetos en la mochila
MAX_WEIGHT = 50  # Peso máximo en la mochila
NBR_ITEMS = 20  # Número total de objetos

Crearemos el conjunto de objetos del que podremos escoger cuáles meter en la mochila.

In [ ]:
# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-tuple.

items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))  # (peso, valor)

Las siguientes líneas crean la población inicial del individuos. Con <code>toolbox.attr_item</code> escogemos un objeto, entre 0 y NBR_ITEMS. La función <code>toolbox.individual</code> itera IND_INIT_SIZE veces para ir añadiendo objetos al individuo. Observa que los individuos pueden tener IND_INIT_SIZE objetos o menos. Eso se debe a que si aleatoriamente se vuelve a escoger el mismo objeto, éste no se repite dentro del conjunto (set). Por último, <code>toolbox.population</code>  crea una función para generar una lista de individuos.

In [ ]:
toolbox = base.Toolbox()
toolbox.register("attr_item", random.randrange, NBR_ITEMS)  # Objeto a escoger, entre 0 y NBR_ITEMS-1
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Evaluamos un individuo como la suma total del peso y valor de sus objetos.

In [ ]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

Para cruzar dos individuos y generar otros dos nuevos podemos usar las funciones de **intersección** y **diferencia simétrica** propias de los conjuntos. Por ejemplo, si tuviéramos los individuos {16, 9, 18, 3} y {3, 4, 13, 17, 18} los individuos resultantes serían {3, 18} (intersección) y {4, 9, 13, 16, 17} (diferencia).

In [ ]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type

    ind1.intersection_update(ind2)
    ind2.symmetric_difference_update(temp)
    
    return ind1, ind2

In [ ]:
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

In [ ]:
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [ ]:
def main():
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    #algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

In [ ]:
pop, stats, hof = main()

# Hall of fame


In [ ]:
hof.items

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for v in items.values():
    x.append(v[0])
    y.append(v[1])


plt.scatter(x, y)
plt.show()

# Frente de Pareto

In [ ]:
x = []
y = []

for hof_item in hof.items:
    if len(hof_item) > 0:
        weight = 0
        value = 0
        for i in hof_item:
            w, v = items[i]
            weight += w
            value += v
        x.append(weight)
        y.append(value)
            
plt.xlabel("Weight")
plt.ylabel("Value")
plt.scatter(x, y)
plt.show()